# CrewAI with Ollama

This Jupyter notebook demonstrates the use of CrewAI in conjunction with the Ollama language model. 

The notebook outlines the creation of various agents and tasks for content preparation, specifically for a YouTube channel focused on topics such as AI, Machine Learning, and Neural Networks.

[Video of the Live Session](https://youtu.be/5hzFOYjH6XI)

[CrewAI](https://crewai.com/) | [Ollama](https://ollama.com/) | [LangChain](https://langchain.com/) | [Agentops](https://agentops.ai/)


#### Install the required packages

In [14]:
%%capture
%pip install -U crewai langchain langchain_community arxiv wikipedia
%pip install -U agentops

This cell installs the necessary packages for the notebook, including crewai, langchain, langchain_community, arxiv, wikipedia.

The %%capture command suppresses the output of the installation process.

## Tasks

In [17]:
from crewai import Agent, Task
from textwrap import dedent
import os
from langchain_community.chat_models.ollama import ChatOllama
from langchain.prompts import ChatPromptTemplate, PromptTemplate
from langchain_core.output_parsers.string import StrOutputParser

# optional
from agentops.langchain_callback_handler import LangchainCallbackHandler as AgentOpsLangchainCallbackHandler
agentops_handler = AgentOpsLangchainCallbackHandler(api_key=os.environ.get('AGENTOPS_API_KEY'), tags=['CrewAI Example'])

from IPython.display import display, Markdown

llm = ChatOllama(model="llama3:latest",
    #callbacks=[agentops_handler],
    n_ctx=4098
)


In [18]:
# Example chain 
#prompt_contribute = PromptTemplate.from_template("""\
#You are a {Role}; 
#{BackStory}
#Your goal is :{Goal}
#Using {Tools}
#""")
#chain  = prompt_contribute | llm | StrOutputParser()
#chain.invoke(""")


This cell imports the necessary libraries and modules. 

These include CrewAI's Agent and Task, dedent from textwrap, various components from langchain, and display utilities from IPython.

In [50]:

class ContentPreparationTasks():
    def research_task(self, agent, topics, subject):
        return Task(
            description=dedent(f"""\
                Conduct comprehensive research on each topic already covered by my channel. Gather information on recent
                news, trends and research papers about the Channel Topic.
                Identify potential gaps and new directions for future content.
                        
                Topics: {topics}
                Channel Subject: {subject}"""),
            expected_output=dedent("""\
                A detailed report summarizing new topics and directions to consider
                for new content relevant to the channel subject."""),
            async_execution=False,
            agent=agent, 
        )

    def outline_creation_task(self, agent, new_topic, subject):
        return Task(
            description=dedent(f"""\
                Analyze the current industry trends, challenges, and opportunities
                relevant to the channel subject. Consider research papers, reports, recent
                developments, and expert opinions to provide a comprehensive
                overview of the {new_topic} for my channel about: {subject}.
                Identify major trends, potential challenges, and future advancements."""),
            expected_output=dedent(f"""\
                An insightful and engaging outline for content on my channel about '{subject}' that identifies major trends, potential
                challenges, and future advancements about {new_topic}."""),
            async_execution=False,
            agent=agent, 
        )
	
    def cross_reference_content_task(self, agent):
        return Task(
            description=dedent("""\
                Analyze the current topics and the outline of the topic to create a comprehensive
                list of cross-references, and connections between the new topics and the existing ones.
                This will help in creating a cohesive and well-structured narrative.

                Topics: {topics}
                New Topic: {new_topic}
                New Outline: {new_outline}
            """),
                
            expected_output=dedent("""\
                An comprehensive list of cross-references and connections between the new topics.
                """),
            async_execution=False,
            agent=agent, 
        )

    def script_writing_task(self, agent):
        return Task(
            description=dedent("""\
                Develop a detailed script based on the provided outline. Ensure the script
                is engaging, informative, and aligns with the overall tone and style of the channel.
                Incorporate any relevant research findings and expert opinions to add depth and credibility.
                
                ##Topic 
                {new_topic}
                
                ## Outline
                {outline}

                ## Channel Subject
                {subject}
            """),
            expected_output=dedent("""\
                A complete and polished script ready for production, including all necessary dialogue, transitions,
                and references to research and expert opinions, using markdown format."""),
            async_execution=False,
            agent=agent,
        )

    def visual_asset_task(self, agent):
        return Task(
            description=dedent("""\
                Identify and create visual assets required for the video based on the script.
                This includes images, graphics, charts, and any other visual elements that
                will enhance the storytelling and provide clear, engaging content for viewers.

                Script: {script}
                Channel Subject: {subject}
            """),
            expected_output=dedent("""\
                A collection of prompts to create visual assets, including images, graphics, charts, and other visual elements, ready to be
                incorporated into the video production process."""),
            async_execution=False,
            agent=agent,
            
        )
    
    def create_twitter_post_task(self, agent, video_url):
        return Task(
            description=dedent(f"""\
                Create an engaging Twitter post to promote the newly published video. The post should include a compelling
                description, relevant hashtags, and a link to the video. 

                Video URL: {video_url}
                Channel Subject: {{subject}}
            """),
            expected_output=dedent("""\
                A ready-to-publish Twitter post that includes an engaging description, relevant hashtags, and a link to the video."""),
            async_execution=False,
            agent=agent
        )

    def create_linkedin_post_task(self, agent, video_url):
        return Task(
            description=dedent(f"""\
                Create an engaging LinkedIn post to promote the newly published video. The post should include a professional
                description, relevant hashtags, and a link to the video. Consider the LinkedIn audience and tailor the content
                to suit a professional network.

                Video URL: {video_url}
                Channel Subject: {{subject}}
            """),
            expected_output=dedent("""\
                A ready-to-publish LinkedIn post that includes a professional description, relevant hashtags, and a link to the video."""),
            async_execution=False,
            agent=agent
        )

    

This class defines various tasks related to content preparation, such as research, outline creation, script writing, visual asset creation, and social media promotion.

## Tools

### Arxiv Research Task

In [51]:
"""Tool for the Arxiv API."""

from typing import Optional, Type

from langchain_core.callbacks import CallbackManagerForToolRun
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_core.tools import BaseTool

from langchain_community.utilities.arxiv import ArxivAPIWrapper


class ArxivInput(BaseModel):
    """Input for the Arxiv tool."""

    query: str = Field(description="search query to look up")
    max_results: int = Field(description="maximum number of results to return", default=5)


# you have ArxivQueryRun: taking as arguments: query and max_results
#"A wrapper around Arxiv.org "
#"Useful for when you need to answer questions about Artificial Inteligence, Machine Physics, Mathematics, "
#"Computer Science, Quantitative Biology, Quantitative Finance, Statistics, "
#"Electrical Engineering, and Economics "
#"from scientific articles on arxiv.org. "
#"Input should be a search query using parameter query."

# param: query: str = "cat=cs.AI and dying neurons"
# param: max_results: int = 5



class ArxivQueryRun(BaseTool):
    """Tool that searches the Arxiv API."""

    name: str = "arxiv"
    description: str = (
        "A wrapper around Arxiv.org "
        "Useful for when you need to answer questions about Physics, Mathematics, "
        "Computer Science, Quantitative Biology, Quantitative Finance, Statistics, "
        "Electrical Engineering, and Economics "
        "from scientific articles on arxiv.org. "
        "Input should be a search query using parameter query."
    )
    api_wrapper: ArxivAPIWrapper = Field(default_factory=ArxivAPIWrapper)
    args_schema: Type[BaseModel] = ArxivInput

    def _run(
        self,
        query: str,
        run_manager: Optional[CallbackManagerForToolRun] = None,
    ) -> str:
        """Use the Arxiv tool."""
        return self.api_wrapper.run(query)
arxiv = ArxivQueryRun() 

docs = arxiv.run("cat=cs.AI and dying neurons")
docs
display(Markdown(f"## Research Papers on the Topic\n\n{docs}"))

## Research Papers on the Topic

Published: 2020-10-21
Title: Dying ReLU and Initialization: Theory and Numerical Examples
Authors: Lu Lu, Yeonjong Shin, Yanhui Su, George Em Karniadakis
Summary: The dying ReLU refers to the problem when ReLU neurons become inactive and
only output 0 for any input. There are many empirical and heuristic
explanations of why ReLU neurons die. However, little is known about its
theoretical analysis. In this paper, we rigorously prove that a deep ReLU
network will eventually die in probability as the depth goes to infinite.
Several methods have been proposed to alleviate the dying ReLU. Perhaps, one of
the simplest treatments is to modify the initialization procedure. One common
way of initializing weights and biases uses symmetric probability
distributions, which suffers from the dying ReLU. We thus propose a new
initialization procedure, namely, a randomized asymmetric initialization. We
prove that the new initialization can effectively prevent the dying ReLU. All
parameters required for the new initialization are theoretically designed.
Numerical examples are provided to demonstrate the effectiveness of the new
initialization procedure.

Published: 2024-03-12
Title: Maxwell's Demon at Work: Efficient Pruning by Leveraging Saturation of Neurons
Authors: Simon Dufort-Labbé, Pierluca D'Oro, Evgenii Nikishin, Razvan Pascanu, Pierre-Luc Bacon, Aristide Baratin
Summary: When training deep neural networks, the phenomenon of $\textit{dying
neurons}$ $\unicode{x2013}$units that become inactive or saturated, output zero
during training$\unicode{x2013}$ has traditionally been viewed as undesirable,
linked with optimization challenges, and contributing to plasticity loss in
continual learning scenarios. In this paper, we reassess this phenomenon,
focusing on sparsity and pruning. By systematically exploring the impact of
various hyperparameter configurations on dying neurons, we unveil their
potential to facilitate simple yet effective structured pruning algorithms. We
introduce $\textit{Demon Pruning}$ (DemP), a method that controls the
proliferation of dead neurons, dynamically leading to network sparsity.
Achieved through a combination of noise injection on active units and a
one-cycled schedule regularization strategy, DemP stands out for its simplicity
and broad applicability. Experiments on CIFAR10 and ImageNet datasets
demonstrate that DemP surpasses existing structured pruning techniques,
showcasing superior accuracy-sparsity tradeoffs and training speedups. These
findings suggest a novel perspective on dying neurons as a valuable resource
for efficient model compression and optimization.

Published: 2017-12-07
Title: Solving internal covariate shift in deep learning with linked neurons
Authors: Carles Roger Riera Molina, Oriol Pujol Vila
Summary: This work proposes a novel solution to the problem of internal covariate
shift and dying neurons using the concept of linked neurons. We define the
neuron linkage in terms of two constraints: first, all neuron activations in
the linkage must have the same operating point. That is to say, all of them
share input weights. Secondly, a set of neurons is linked if and only if there
is at least one member of the linkage that has a non-zero gradient in regard to
the input of the activation function. This means that for any input in the
activation function, there is at least one member of the linkage that operates
in a non-flat and non-zero area. This simple change has profound implications
in the network learning dynamics. In this article we explore the consequences
of this proposal and show that by using this kind of units, internal covariate
shift is implicitly solved. As a result of this, the use of linked neurons
allows to train arbitrarily large networks without any architectural or
algorithmic trick, effectively removing the need of using re-normalization
schemes such as Batch Normalization, which leads to halving the required
training time. It also solves the problem of the need for stan

### Wikipedia Task

In [52]:
from langchain.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

wikipedia = WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper(top_k_results=5))

wikipedia_docs = wikipedia.run("backpropagation algorithms")
display(Markdown(f"## Research Papers on the Topic\n\n{wikipedia_docs}"))

## Research Papers on the Topic

Page: Backpropagation
Summary: In machine learning, backpropagation is a gradient estimation method used to train neural network models. The gradient estimate is used by the optimization algorithm to compute the network parameter updates.
It is an efficient application of the Leibniz chain rule (1673) to such networks. It is also known as the reverse mode of automatic differentiation or reverse accumulation, due to Seppo Linnainmaa (1970). The term "back-propagating error correction" was introduced in 1962 by Frank Rosenblatt, but he did not know how to implement this, even though Henry J. Kelley had a continuous precursor of backpropagation already in 1960 in the context of control theory.
Backpropagation computes the gradient of a loss function with respect to the weights of the network for a single input–output example, and does so efficiently, computing the gradient one layer at a time, iterating backward from the last layer to avoid redundant calculations of intermediate terms in the chain rule; this can be derived through dynamic programming. Gradient descent, or variants such as stochastic gradient descent, are commonly used.
Strictly the term backpropagation refers only to the algorithm for computing the gradient, not how the gradient is used; but the term is often used loosely to refer to the entire learning algorithm – including how the gradient is used, such as by stochastic gradient descent. In 1986 David E. Rumelhart et al. published an experimental analysis of the technique. This contributed to the popularization of backpropagation and helped to initiate an active period of research in multilayer perceptrons.

Page: Backpropagation through time
Summary: Backpropagation through time (BPTT) is a gradient-based technique for training certain types of recurrent neural networks. It can be used to train Elman networks. The algorithm was independently derived by numerous researchers.



Page: Neural backpropagation
Summary: Neural backpropagation is the phenomenon in which, after the action potential of a neuron creates a voltage spike down the axon (normal propagation), another impulse is generated from the soma and propagates towards the apical portions of the dendritic arbor or dendrites (from which much of the original input current originated).  In addition to active backpropagation of the action potential, there is also passive electrotonic spread. While there is ample evidence to prove the existence of backpropagating action potentials, the function of such action potentials and the extent to which they invade the most distal dendrites remain highly controversial.

Page: Almeida–Pineda recurrent backpropagation
Summary: Almeida–Pineda recurrent backpropagation is an extension to the backpropagation algorithm that is applicable to recurrent neural networks. It is a type of supervised learning. It was described somewhat cryptically in Richard Feynman's senior thesis, and rediscovered independently in the context of artificial neural networks by both Fernando Pineda and Luis B. Almeida.
A recurrent neural network for this algorithm consists of some input units, some output units and eventually some hidden units.
For a given set of (input, target) states, the network is trained to settle into a stable activation state with the output units in the target state, based on a given input state clamped on the input units.

Page: History of artificial neural networks
Summary: Artificial neural networks (ANNs) are models created using machine learning to perform a number of tasks. Their creation was inspired by neural circuitry. While some of the computational implementations ANNs relate to earlier discoveries in mathematics, the first implementation of ANNs was by psychologist Frank Rosenblatt, who developed the perceptron. Little research was conducted on ANNs in the 1970s and 1980s, with the AAAI calling that period an "AI winter". 
Later, advances in hardware and the development of the backpropagation algorithm as well a

### Setting up tools for agents


In [53]:
from langchain.agents import load_tools
tools = load_tools(
    ["wikipedia"],
) 

tools = tools + [arxiv]
tools

[WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper(wiki_client=<module 'wikipedia' from 'C:\\Users\\sebastiendg\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\wikipedia\\__init__.py'>, top_k_results=3, lang='en', load_all_available_meta=False, doc_content_chars_max=4000)),
 ArxivQueryRun()]

## Agents

In [54]:
class ContentPreparationAgents:
    def __init__(self, llm):
        self.llm = llm

    # Define the agents involved in the content creation process
    def create_researcher(self,tools=[]):
        return Agent(
            role="Research Agent",

            backstory="""\
            Ray was a former investigative journalist who has a knack for uncovering the latest trends and insights.
            After years of digging deep into stories, Ray transitioned into the digital content world, leveraging his skills to help content creators stay ahead of the curve.""",
            
            goal="To provide comprehensive and up-to-date research that helps shape the direction of the channel’s content.",
            llm=self.llm,
            
            tools=tools,
            allow_delegation=False,
			verbose=True
        )

    def create_outline_writer(self,tools=[]):
        return Agent(
            role="Outline Creation Agent",
            backstory="""\
            Olivia was once a strategy consultant for major media firms, known for her ability to synthesize complex information into clear, actionable plans.
            She now uses her talents to help content creators map out their content effectively.""",

            goal="To create detailed and engaging outlines that guide the content creation process seamlessly.",
            llm=self.llm,
            tools=tools,
            allow_delegation=False,
			verbose=True
    )
    
    def create_content_agent(self, tools=[]):
        return Agent(
            role="Content Integration Agent",

            backstory="""\
            Chris has a background in library sciences and data management.
            His expertise in organizing and cross-referencing large volumes of information makes him perfect for ensuring new content fits well within existing frameworks.""",
            
            goal="To build a cohesive narrative by connecting new content with existing topics.",

            tools=tools,
            llm=self.llm,
            allow_delegation=False,
			verbose=True
        )

    def create_script_writer(self,tools=[]):
        return Agent(
        role="Script Writing Agent",
        backstory="""\
            Sam is a former screenwriter with experience in both film and digital media.
            His storytelling skills ensure that every piece of content is engaging and informative.""",

        goal="To craft scripts that are compelling and aligned with the channel's voice and vision.",
        tools=tools,
        llm=self.llm,
        allow_delegation=False,
		verbose=True
    )

    def create_visual_asset_creator(self,tools=[]):
        return Agent(
        role="Visual Asset Creation Agent",

        backstory="""\
        Vicky is a graphic designer and visual artist who has worked with numerous brands to create eye-catching visuals.
        Her ability to translate concepts into compelling visuals and their prompt is unparalleled.""",
        
        goal="To create visual assets that enhance the storytelling of the content.",
        tools=tools,
        llm=self.llm,
        allow_delegation=False,
		verbose=True
    )

    def create_social_media_agent(self,tools=[]):
        return Agent(
        role="Social Media and Community Engagement Agent",

        backstory="""\
        Casey is a social media strategist who has successfully managed online communities for various brands.
        With expertise in crafting engaging posts and interacting with audiences, Casey ensures that the content reaches and resonates with a wide audience.""",

        goal="To create and schedule engaging posts for Twitter and LinkedIn that promote the new video content effectively.",
        tools=tools,
        llm=self.llm,
        allow_delegation=False,
		verbose=True
    )

This class defines various agents involved in content preparation, such as content agent, 
outline writer, researcher, script writer, visual asset creator, and social media agent.

## Crew

In [55]:
from crewai import Crew

tasks = ContentPreparationTasks()
agents = ContentPreparationAgents(llm)

content_agent = agents.create_content_agent(tools)
outline_writer = agents.create_outline_writer(tools)
researcher = agents.create_researcher(tools)
script_writer = agents.create_script_writer(tools)
visual_asset_creator = agents.create_visual_asset_creator(tools)
social_media_agent = agents.create_social_media_agent(tools)

youtuber_crew = Crew(
    agents=[
        researcher,
        content_agent,
        outline_writer,
        researcher,
        script_writer,
        visual_asset_creator,
        social_media_agent
    ], 
    tasks=[
        tasks.research_task(researcher, "AI, Machine Learning, Neural Networks", "Artificial Intelligence"),

        tasks.outline_creation_task(outline_writer, "Backpropagation Algorithms", "Artificial Intelligence"),
            
        tasks.cross_reference_content_task(content_agent),

        tasks.script_writing_task(script_writer),

        tasks.visual_asset_task(visual_asset_creator),

        tasks.create_twitter_post_task(social_media_agent, 
            "https://youtube.com/video"),
        tasks.create_linkedin_post_task(social_media_agent,
             "https://youtube.com/video")
    ],
    manager=llm
)

This section initializes the tasks and agents, assigns specific tasks to the respective agents, and creates a crew of agents managed by the language model.


In [56]:
youtuber_crew.kickoff()



> Entering new CrewAgentExecutor chain...


This command starts the process, activating the agents to perform their assigned tasks.